In [1]:
from zellij.strategies.tools.geometry_new import Hypersphere, Hypercube, Section
from zellij.core import ArrayVar, FloatVar, Loss
from zellij.utils.benchmarks import himmelblau
from zellij.utils.converters import FloatMinmax, FloatBinning, ArrayConverter, FloatBinary, FloatBinaryStochastic, ArrayBinaryBest, Basic
from zellij.strategies.tools.measurements import SigmaInf, Sigma2, Level, Radius
from zellij.strategies.tools.tree_search_new import (
    Depth_first_search,
    Breadth_first_search,
    Best_first_search,
    Beam_search,
    Epsilon_greedy_search,
    Potentially_Optimal_Rectangle,
    Locally_biased_POR,
    Adaptive_POR,
    Soo_tree_search,
    Move_up,
)

from zellij.strategies.tools.scoring import (
    Min,
    Median,
    Mean,
    Std,
    Distance_to_the_best,
    Distance_to_the_best_centered,
    Belief,
)

import numpy as np
import logging
logging.root.setLevel(logging.DEBUG)

In [2]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))
vshaped = lambda x: np.abs(np.tanh(x))

lf = Loss()(himmelblau)
values = ArrayVar(
    FloatVar("float_1", -5 , 5, converter=FloatMinmax()),
    FloatVar("float_2", -5, 5, converter=FloatMinmax()),
    converter=ArrayBinaryBest(loss=lf),
)

sp = Hypersphere(values, lf, measure=Level(), converter=Basic())
ts = Depth_first_search(open=sp, max_depth=3, Q=2)
sc = Min()

In [3]:
points = sp.random_point(100)
cp = []
for p in points:
    cp.append(values.converter.convert(p))
X,Y = lf(cp)

In [5]:
points, cp, lf.best_point, lf.best_score

([[1.094817266639641, -0.4596048053345294],
  [-4.326871311079553, -4.227519513592581],
  [2.7039292772606096, -4.509376385917511],
  [2.8932480783420598, -4.448116802094716],
  [0.11693351530001372, -2.8989157643132915],
  [1.0195347662068972, 1.091240438944041],
  [2.2995481619963805, 1.773020051426223],
  [4.178991130641446, 0.9859654199459325],
  [0.4876577078512909, 0.6407679425527224],
  [3.9531104679387212, -3.5634823636618695],
  [4.7836548447452305, -0.8181146106635477],
  [-2.9078710415529594, 0.22765993923541927],
  [4.602031483397566, 2.926904997745483],
  [-4.070430127579288, -1.1817032605762066],
  [3.774074233539377, -1.1427352066154803],
  [2.6494157643939387, -2.8156311111275922],
  [3.2967789711875035, -1.7635015393584306],
  [1.1637904905834802, -4.656729898672953],
  [2.5104570138201403, 4.466233162872662],
  [2.3334002289320743, 4.202693299600025],
  [3.3779083920410216, 2.3211141484925033],
  [0.5071332349644777, -4.976078086863751],
  [-4.8898704879994455, -2.024

In [3]:
ctn, current = ts.get_next()
children = current[0].create_children()
children

[Hypersphere(1,0,0),
 Hypersphere(1,0,1),
 Hypersphere(1,0,2),
 Hypersphere(1,0,3)]

In [4]:
for c in children:
    ts.add(c)
ctn, current = ts.get_next()

In [13]:
current[0]
current[1].radius, current[1].center, current[1].f_id, current[1].measure, current[1].solutions, current[1].losses

(0.20710678118654754, array([0.5       , 0.79289322]), 1, 1, [], [])

In [14]:
print(current[0], current[0].lower, current[0].upper)
points = current[0].center
converted = sp.converter.reverse(points)
points,converted

Hypersphere(1,0,0) [0. 0.] [1. 1.]


(array([0.79289322, 0.5       ]), [2.9289321881345245, 0.0])

In [16]:
X,Y = lf([converted])
X,Y

([[2.9289321881345245, 0.0]], [-24.510255227613385])

In [17]:
cpoints = sp.converter.convert(X)
current[0].add_solutions(cpoints,Y)
current[0].solutions,current[0].losses

([[0.7928932188134524, 0.5]], [-24.510255227613385])

In [19]:
current[0].score = sc(current[0])

In [20]:
children = current[0].create_children()
children

[Hypersphere(2,0,0),
 Hypersphere(2,0,1),
 Hypersphere(2,0,2),
 Hypersphere(2,0,3)]

In [21]:
for c in ts.close:
    print(c.lower, c.upper, c.get_id(), c.score, c.measure)

[0. 0.] [1. 1.] (0, -1, 0) nan 0
[0. 0.] [1. 1.] (1, 0, 0) -24.510255227613385 1
[0. 0.] [1. 1.] (1, 0, 1) nan 1


In [12]:
for c in ts.open:
    print(c.lower, c.upper, c.get_id(), c.score, c.measure)

[0.5 0. ] [1.  0.5] (1, 1, 2) nan 1
[0.5 0.5] [1. 1.] (1, 1, 3) nan 1


In [13]:
for c in children:
    print(c.lower, c.upper, c.get_id(), c.score, c.measure)

[0. 0.] [0.25 0.25] (2, 0, 0) -65.92921119744798 2
[0.   0.25] [0.25 0.5 ] (2, 0, 1) -65.92921119744798 2
[0.25 0.  ] [0.5  0.25] (2, 0, 2) -65.92921119744798 2
[0.25 0.25] [0.5 0.5] (2, 0, 3) -65.92921119744798 2
